In [1]:
import numpy as np 
import pandas as pd 

In [2]:
train_data = pd.read_csv("/kaggle/input/titanic/train.csv")

In [3]:
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")

In [5]:
women = train_data.loc[train_data.Sex == 'female']["Survived"]
rate_women = sum(women)/len(women)

print("% of women who survived:", rate_women)

% of women who survived: 0.7420382165605095


In [6]:
men = train_data.loc[train_data.Sex == 'male']["Survived"]
rate_men = sum(men)/len(men)

print("% of men who survived:", rate_men)

% of men who survived: 0.18890814558058924


In [7]:
print("% of women who survived:", rate_women)
print("% of men who survived:", rate_men)

% of women who survived: 0.7420382165605095
% of men who survived: 0.18890814558058924


In [8]:
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, MinMaxScaler
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectKBest,chi2
import warnings
warnings.filterwarnings('ignore')

In [9]:
train_data.isnull().mean()*100

PassengerId     0.000000
Survived        0.000000
Pclass          0.000000
Name            0.000000
Sex             0.000000
Age            19.865320
SibSp           0.000000
Parch           0.000000
Ticket          0.000000
Fare            0.000000
Cabin          77.104377
Embarked        0.224467
dtype: float64

In [10]:
test_data.isnull().mean()*100

PassengerId     0.000000
Pclass          0.000000
Name            0.000000
Sex             0.000000
Age            20.574163
SibSp           0.000000
Parch           0.000000
Ticket          0.000000
Fare            0.239234
Cabin          78.229665
Embarked        0.000000
dtype: float64

In [11]:
train_data.drop("Cabin", axis = 1, inplace=True)
test_data.drop("Cabin", axis = 1, inplace=True)

In [12]:
si = SimpleImputer()
oe = OrdinalEncoder(handle_unknown="ignore")
ohe = OneHotEncoder(handle_unknown="ignore", sparse=False)

In [13]:
train_data["family"] = train_data["SibSp"]+train_data["Parch"]+1
test_data["family"] = test_data["SibSp"]+test_data["Parch"]+1


In [14]:
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,family
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S,2
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C,2
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S,1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S,2
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,S,1


In [15]:
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,family
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,Q,1
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,S,2
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,Q,1
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,S,1
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,S,3


In [16]:
train_data.drop(["SibSp","Parch"],axis = 1, inplace=True)
test_data.drop(["SibSp","Parch"],axis = 1, inplace=True)


In [17]:
train_data.sample(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,Ticket,Fare,Embarked,family
460,461,1,1,"Anderson, Mr. Harry",male,48.0,19952,26.5500,S,1
463,464,0,2,"Milling, Mr. Jacob Christian",male,48.0,234360,13.0000,S,1
819,820,0,3,"Skoog, Master. Karl Thorsten",male,10.0,347088,27.9000,S,6
681,682,1,1,"Hassab, Mr. Hammad",male,27.0,PC 17572,76.7292,C,1
710,711,1,1,"Mayne, Mlle. Berthe Antonine (""Mrs de Villiers"")",female,24.0,PC 17482,49.5042,C,1


In [18]:
train_data.dropna( subset='Embarked',inplace=True)
test_data.dropna( subset='Embarked',inplace=True)

In [19]:
train_data.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
Ticket           0
Fare             0
Embarked         0
family           0
dtype: int64

In [20]:
train_data.drop(["Name","Ticket","Fare"],axis = 1, inplace=True)
test_data.drop(["Name","Ticket","Fare"],axis = 1, inplace=True)

In [21]:
train_data.sample(5)

,PassengerId,Survived,Pclass,Sex,Age,Embarked,family
266,267,0,3,male,16.0,S,6
123,124,1,2,female,32.5,S,1
28,29,1,3,female,NaN,Q,1
839,840,1,1,male,NaN,C,1
722,723,0,2,male,34.0,S,1


In [22]:
test_data.head()

,PassengerId,Pclass,Sex,Age,Embarked,family
0,892,3,male,34.5,Q,1
1,893,3,female,47.0,S,2
2,894,2,male,62.0,Q,1
3,895,3,male,27.0,S,1
4,896,3,female,22.0,S,3


In [23]:
X_train = train_data.drop(["Survived","PassengerId"],axis = 1)
y_train = train_data["Survived"]
X_test  = test_data.drop("PassengerId",axis = 1)


In [24]:
trf = ColumnTransformer(transformers=[("age_imputation",si,["Age"]),
                                      ("Sex",ohe,["Sex","Embarked"])],remainder="passthrough")

In [25]:
trf2 = SelectKBest(score_func=chi2, k="all")

In [26]:
trf3 = ColumnTransformer([("min_max", MinMaxScaler(),slice(0,4))])

In [27]:
trf1 = RandomForestClassifier()

In [28]:
pipe = Pipeline(steps=[("trf",trf),("trf3",trf3),("trf2",trf2)
               ,("trf1",trf1)])

In [29]:
pipe.fit(X_train,y_train)

Pipeline(steps=[('trf',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('age_imputation',
                                                  SimpleImputer(), ['Age']),
                                                 ('Sex',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  ['Sex', 'Embarked'])])),
                ('trf3',
                 ColumnTransformer(transformers=[('min_max', MinMaxScaler(),
                                                  slice(0, 4, None))])),
                ('trf2',
                 SelectKBest(k='all',
                             score_func=<function chi2 at 0x7f4994c6d990>)),
                ('trf1', RandomForestClassifier())])

In [30]:
trf1.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'sqrt',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [31]:
params = {"trf1__criterion":["entropy","gini","log_loss"],"trf1__max_depth":[1,2,3,4,5]
         ,"trf1__n_estimators":[50,100,150,200,250]
         
         }

In [32]:
pipe.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'trf', 'trf3', 'trf2', 'trf1', 'trf__n_jobs', 'trf__remainder', 'trf__sparse_threshold', 'trf__transformer_weights', 'trf__transformers', 'trf__verbose', 'trf__verbose_feature_names_out', 'trf__age_imputation', 'trf__Sex', 'trf__age_imputation__add_indicator', 'trf__age_imputation__copy', 'trf__age_imputation__fill_value', 'trf__age_imputation__keep_empty_features', 'trf__age_imputation__missing_values', 'trf__age_imputation__strategy', 'trf__age_imputation__verbose', 'trf__Sex__categories', 'trf__Sex__drop', 'trf__Sex__dtype', 'trf__Sex__handle_unknown', 'trf__Sex__max_categories', 'trf__Sex__min_frequency', 'trf__Sex__sparse', 'trf__Sex__sparse_output', 'trf3__n_jobs', 'trf3__remainder', 'trf3__sparse_threshold', 'trf3__transformer_weights', 'trf3__transformers', 'trf3__verbose', 'trf3__verbose_feature_names_out', 'trf3__min_max', 'trf3__min_max__clip', 'trf3__min_max__copy', 'trf3__min_max__feature_range', 'trf2__k', 'trf2__score_func', 'trf1

In [33]:
from sklearn.model_selection import GridSearchCV

In [34]:
grid = GridSearchCV(pipe,params,cv =  5,scoring = "accuracy")

In [35]:
grid.fit(X_train,y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('trf',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('age_imputation',
                                                                         SimpleImputer(),
                                                                         ['Age']),
                                                                        ('Sex',
                                                                         OneHotEncoder(handle_unknown='ignore',
                                                                                       sparse=False),
                                                                         ['Sex',
                                                                          'Embarked'])])),
                                       ('trf3',
                                        ColumnTransformer(transformers=[('min_max',
                                                                         MinMaxScaler(),
                                                                         slice(0, 4, None))])),
                                       ('trf2',
                                        SelectKBest(k='all',
                                                    score_func=<function chi2 at 0x7f4994c6d990>)),
                                       ('trf1', RandomForestClassifier())]),
             param_grid={'trf1__criterion': ['entropy', 'gini', 'log_loss'],
                         'trf1__max_depth': [1, 2, 3, 4, 5],
                         'trf1__n_estimators': [50, 100, 150, 200, 250]},
             scoring='accuracy')

In [36]:
prediction = grid.predict(X_test)

In [37]:
output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': prediction})
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!


In [38]:
output

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0
